In [90]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression,Ridge,Lasso,ElasticNet
from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder,StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score

In [2]:
data=pd.read_csv(r"C:\Users\mdfir\Downloads\data.csv")

In [3]:
data.head()

,house_type,locality,city,area,beds,bathrooms,balconies,furnishing,area_rate,rent
0,"2 BHK Flat for Rent in Oberoi Woods, Goregaon ...",Goregaon East,Mumbai,897.0,2,2,0,Semi-Furnished,134.0,120000.0
1,"1 BHK Flat for Rent in Sapphire Lakeside, Powa...",Powai,Mumbai,490.0,1,1,0,Semi-Furnished,82.0,40000.0
2,1 BHK House for Rent in Mundhwa Pune,Mundhwa,Pune,550.0,1,1,0,Unfurnished,22.0,12000.0
3,"2 BHK Flat for Rent in Hingna, Nagpur",Hingna,Nagpur,1000.0,2,2,0,Unfurnished,8.0,8000.0
4,1 BHK Flat for Rent in Unique Star Harsh Vihar...,Mira Road,Mumbai,595.0,1,1,0,Unfurnished,25.0,15000.0


In [4]:
data=data.drop(columns=["house_type"])

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7691 entries, 0 to 7690
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   locality    7691 non-null   object 
 1   city        7691 non-null   object 
 2   area        7691 non-null   float64
 3   beds        7691 non-null   int64  
 4   bathrooms   7691 non-null   int64  
 5   balconies   7691 non-null   int64  
 6   furnishing  7691 non-null   object 
 7   area_rate   7691 non-null   float64
 8   rent        7691 non-null   float64
dtypes: float64(3), int64(3), object(3)
memory usage: 540.9+ KB


In [6]:
data["furnishing"].value_counts()

furnishing
Semi-Furnished    3428
Unfurnished       2662
Furnished         1601
Name: count, dtype: int64

In [100]:
x=data.drop(columns=["rent"])
y=data["rent"]

In [103]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=.2,random_state=42)

In [106]:
onehot=["locality","city"]
ordinal=["furnishing"]
cat=["area","beds","bathrooms","balconies","area_rate"]

In [108]:
one=Pipeline([
("onehotencoding",OneHotEncoder(handle_unknown="ignore",sparse_output=False))
])

In [110]:
ordi=Pipeline([
("ordinalencoder",OrdinalEncoder(categories=[["Unfurnished","Semi-Furnished","Furnished"]]))
   
])



In [112]:
scaler=Pipeline([
("scaler",StandardScaler())
  ])

In [114]:
preprocessing=ColumnTransformer([
("onehot",one,onehot),
    ("ordinal",ordi,ordinal),
    ("scaler",scaler,cat)

    
],remainder="passthrough")

In [116]:
pip=Pipeline([
("preprocessing",preprocessing),
   ("model",LinearRegression())

    
])

In [120]:
pip.fit(x_train,y_train)

,steps,"[('preprocessing', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('onehot', ...), ('ordinal', ...), ...]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [121]:
pred=pip.predict(x_test)

In [122]:
score=r2_score(y_test,pred)

In [123]:
score

0.5584587078843846

# ridge regression 

In [129]:
pip_r=Pipeline([
("preprocessing",preprocessing),
   ("model",Ridge(alpha=2,max_iter=500))
])

In [131]:
pip_r.fit(x_train,y_train)

,steps,"[('preprocessing', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('onehot', ...), ('ordinal', ...), ...]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [132]:
y_pred=pip_r.predict(x_test)

In [133]:
score=r2_score(y_test,y_pred)

In [137]:
score

0.5953546438173626

In [139]:
pip_r.named_steps['model'].coef_



array([-4182.82392358,  8970.41666802,  8064.48291348, ...,
       19840.4895132 , -1746.9550181 , 36376.91271545])

In [141]:
pip_r.named_steps['model'].intercept_

50787.43936837981

# Lasso Regression

In [144]:
pip_l=Pipeline([
("preprocessing",preprocessing),
   ("model",Lasso(alpha=1,max_iter=500))
])

In [146]:
pip_l.fit(x_train,y_train)

C:\Users\mdfir\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.247e+11, tolerance: 6.145e+09
  model = cd_fast.enet_coordinate_descent(


,steps,"[('preprocessing', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('onehot', ...), ('ordinal', ...), ...]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [147]:
l_pred=pip_l.predict(x_test)

In [148]:
score=r2_score(y_test,l_pred)

In [149]:
score

0.5722077809240292

# ElasticNet Regression 

In [185]:
pip_e=Pipeline([
("preprocessing",preprocessing),
   ("model",ElasticNet(alpha=.01,l1_ratio=.4))
])

In [187]:
pip_e.fit(x_train,y_train)

,steps,"[('preprocessing', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('onehot', ...), ('ordinal', ...), ...]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [188]:
pred_e=pip_e.predict(x_test)

In [189]:
score=r2_score(y_test,pred_e)

In [190]:
score

0.5569030546252928